In [1]:
from pathlib import Path
import os

yr = 2024
d = 15

inp_path = os.path.join(Path(os.path.abspath("")).parents[2], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))

with open(inp_path, 'r') as file:
    inp = file.readlines()

In [2]:
class Warehouse:

  def __init__(self, layout):
    self.layout = layout
    self.max_i = max([c[0] for c in self.layout.keys()])
    self.max_j = max([c[1] for c in self.layout.keys()])
    self.loc = self.get_loc()

  def move(self, dir):
    if self.can_move(dir):
      n = 1
      move_map = {}
      while self.layout[add_tup(self.loc, multiply_tup(dir, n))] in 'O[]':
        move_map[add_tup(self.loc, multiply_tup(dir, n+1))] = self.layout[add_tup(self.loc, multiply_tup(dir, n))]
        n += 1
      self.layout.update(move_map)
      self.layout[add_tup(self.loc, dir)] = '@'
      self.layout[self.loc] = '.'
      self.loc = add_tup(self.loc, dir)
    return self

  def can_move(self, dir):
    for n in range(1, (self.max_j if dir[0]==0 else self.max_i)+1):
      s = self.layout.get(add_tup(self.loc, multiply_tup(dir, n)), "#")
      if s == '.':
        return True
      elif s == '#':
        return False
    return False


  def get_loc(self):
    return [k for k in self.layout.keys() if self.layout[k]=='@'][0]

  def __str__(self):

    return '\n'.join([''.join([self.layout.get((i,j), '') 
                               for j in range(self.max_j+1)]) for i in range(self.max_i+1)])


def format_input(inp):
  dirs = {'^': (-1, 0), 
          'v': (1, 0), 
          '<': (0, -1), 
          '>': (0, 1)}

  m_ = [l.strip() for l in inp if l.startswith('#')]
  moves = ''.join([l.strip() for l in inp if any([c in l for c in dirs.keys()])])
  m = {}
  for i, l in enumerate(m_):
    for j, c in enumerate(l):
      m[(i, j)] = c
  moves = [dirs[c] for c in moves]
  return Warehouse(m), moves




class Thicc_Warehouse(Warehouse):

  def __init__(self, layout):
    self.layout = thiccen(layout)
    self.max_i = max([c[0] for c in self.layout.keys()])
    self.max_j = max([c[1] for c in self.layout.keys()])
    self.loc = self.get_loc()

  
  def move(self, dir):
    if dir[0] == 0 or self.layout[add_tup(self.loc, dir)] not in ['[', ']']:
      return super().move(dir)

    def get_all_pushed_thicc_boxes(loc, dir):
      if self.layout[add_tup(loc, dir)] == '[':
        other_loc = add_tup(add_tup(loc, dir), (0,1))
        return [add_tup(loc, dir), other_loc]  + get_all_pushed_thicc_boxes(add_tup(loc, dir), dir) + get_all_pushed_thicc_boxes(other_loc, dir)
      elif self.layout[add_tup(loc, dir)] == ']':
        other_loc = add_tup(add_tup(loc, dir), (0,-1))
        return [add_tup(loc, dir), other_loc]  + get_all_pushed_thicc_boxes(add_tup(loc, dir), dir) + get_all_pushed_thicc_boxes(other_loc, dir)
      return []
    

    if self.can_move(dir):
      to_move = get_all_pushed_thicc_boxes(self.loc, dir)
      dests = [add_tup(l, dir) for l in to_move]
      cur_items = [self.layout[tm] for tm in to_move]
      for d, ci  in zip(dests, cur_items):
        self.layout[d] = ci
      for tm in set(to_move).difference(set(dests)):
        self.layout[tm] = '.'
      self.layout[add_tup(self.loc, dir)] = '@'
      self.layout[self.loc] = '.'
      self.loc = add_tup(self.loc, dir)
    return self
      

  def can_move(self, dir):
    if dir[0] == 0 or self.layout[add_tup(self.loc, dir)] not in ['[', ']']:
      return super().can_move(dir)

    def check_thicc_box_vert(loc, dir):

      if self.layout[add_tup(loc, dir)] == '[':
        other_loc = add_tup(add_tup(loc, dir), (0,1))
        return check_thicc_box_vert(add_tup(loc, dir), dir) and check_thicc_box_vert(other_loc, dir)
      elif self.layout[add_tup(loc, dir)] == ']':
        other_loc = add_tup(add_tup(loc, dir), (0,-1))
        return check_thicc_box_vert(add_tup(loc, dir), dir) and check_thicc_box_vert(other_loc, dir)
      elif self.layout[add_tup(loc, dir)] == '.':
        return True
      elif self.layout[add_tup(loc, dir)] == '#':
        return False
      raise Exception('Wat')
    
    return check_thicc_box_vert(self.loc, dir)  

def add_tup(a, b):
  return (a[0]+b[0], a[1]+b[1])

def multiply_tup(a, n):
  return (a[0]*n, a[1]*n)


def thiccen(layout):
  new_layout = {}
  for loc, item in layout.items():
    if item == '#':
      new_items = '##'
    elif item == '.':
      new_items = '..'
    elif item == 'O':
      new_items = '[]'
    elif item == '@':
      new_items = '@.'
    new_layout[(loc[0], loc[1]*2)] = new_items[0]
    new_layout[(loc[0], loc[1]*2+1)] = new_items[1]
  return new_layout

In [3]:
def make_moves(warehouse, moves):
  import copy
  warehouse_ = copy.deepcopy(warehouse)
  for move in moves:
    warehouse_ = warehouse_.move(move)
  return warehouse_

def sum_gps_coords(warehouse, box='O'):
  return sum(map(lambda x: x[0]*100 + x[1], [k for k, v in warehouse.layout.items() if v==box]))


In [4]:
import time

t = time.time()

formatted_input = format_input(inp)

print(sum_gps_coords(make_moves(formatted_input[0], formatted_input[1])))
print(sum_gps_coords(make_moves(Thicc_Warehouse(formatted_input[0].layout), formatted_input[1]), box='['))

print('\nRUNTIME: ', time.time()-t)

1499739
1522215

RUNTIME:  0.14772725105285645
